In [ ]:
!pip install nfl_data_py

In [2]:
import nfl_data_py as nfl
import pandas as pd

In [3]:
nflpbp_df = nfl.import_pbp_data([2024])

2024 done.
Downcasting floats.


In [4]:
def get_field_goal_data(df: pd.DataFrame, offense, defense: str) -> pd.DataFrame:
  """Returns a DataFrame of field goal data for the given play-by-play data.

  Finds all games where a field goal was made in the 4th quater which put the
  field goal kicker's team ahead of the opponent's score where the kicker's team
  ultimately won the game.

  Returns:
    A DataFrame with:
    - The kicker's name
    - The win probability added for the made field goal
    - The number of seconds remaining in the game
  """
  out_df = df.loc[
    # Filter events by field goals made ...
    (df['field_goal_result'] == 'made') &
    # in the 4th quarter ...
    (df['qtr'] == 4) &
    (df['posteam_type'] == offense) &
    # where the kicker's team is trailing or tied before the field goal ...
    (df['posteam_score'] <= df[f'total_{defense}_score']) &
    # and leading or tied after the field goal is made ...
    (df['posteam_score_post'] >= df[f'total_{defense}_score']) &
    # and the kicker's team ultimately wins the game
    (df[f'{offense}_score'] > df[f'{defense}_score'])
    ][['kicker_player_name', f'{offense}_team', 'wpa', 'game_seconds_remaining']]
  out_df = out_df.rename(columns={f'{offense}_team': 'team'})
  return out_df

In [5]:
home_fgs_df = get_field_goal_data(nflpbp_df, 'home', 'away')
away_fgs_df = get_field_goal_data(nflpbp_df, 'away', 'home')
fgs_df = pd.concat([home_fgs_df, away_fgs_df]).reset_index(drop=True)

In [6]:
kicker_gb = fgs_df.groupby('kicker_player_name')
# The kicker's team
team_s = kicker_gb['team'].unique()
# Number of made field goals by kicker
num_fgs_s = kicker_gb.size()
num_fgs_s.name = 'num_fgs'
# Average win probability added for each made field goal
wpa_avg_s = kicker_gb['wpa'].median()
wpa_avg_s.name = 'avg_wpa'
# Average seconds remaining in the game for each made field goal
secs_remain_s = kicker_gb['game_seconds_remaining'].mean()
secs_remain_s.name = 'avg_secs_remain'
fg_rank_df = pd.concat([team_s, num_fgs_s, wpa_avg_s, secs_remain_s], axis=1)

In [7]:
fg_rank_df.sort_values(by=['num_fgs', 'avg_wpa', 'avg_secs_remain'], ascending=[False, False, True])

,team,num_fgs,avg_wpa,avg_secs_remain
kicker_player_name,,,,
J.Bates,[DET],5,0.160093,70.199997
J.Karty,[LA],4,0.034216,541.000000
C.Ryland,[ARI],3,0.303390,34.333332
J.Sanders,[MIA],3,0.288324,94.000000
H.Butker,[KC],3,0.107806,193.000000
M.Wright,[KC],3,0.084477,462.333344
Y.Koo,[ATL],2,0.414929,4.000000
B.McManus,[GB],2,0.261047,2.500000
T.Bass,[BUF],2,0.151356,118.000000
